# SLCSP

## Calculate the second lowest cost silver plan

## Problem

You've been asked to determine the second lowest cost silver plan (SLCSP) for
a group of ZIP codes.

## Task

You've been given a CSV file, `slcsp.csv`, which contains the ZIP codes in the
first column. Fill in the second column with the rate (see below) of the
corresponding SLCSP and emit the answer on `stdout` using the same CSV format as
the input. Write your code in your best programming language.

### Expected output

The order of the rows in your answer as emitted on stdout must stay the same as how they
appeared in the original `slcsp.csv`. The first row should be the column headers: `zipcode,rate`
The remaining lines should output unquoted values with two digits after the decimal
place of the rates, for example: `64148,245.20`.

It may not be possible to determine a SLCSP for every ZIP code given; for example, if there is only one silver plan available, there is no _second_ lowest cost plan. Check for cases where a definitive answer cannot be found and leave those cells blank in the output (no quotes or zeroes or other text). For example, `40813,`.

## Additional information

The SLCSP is the so-called "benchmark" health plan in a particular area. It's
used to compute the tax credit that qualifying individuals and families receive
on the marketplace. It's the second lowest rate for a silver plan in the rate area.

For example, if a rate area had silver plans with rates of `[197.3, 197.3, 201.1, 305.4, 306.7, 411.24]`, the SLCSP for that rate area would be `201.1`,
since it's the second lowest rate in that rate area.

A plan has a "metal level", which can be either Bronze, Silver, Gold, Platinum,
or Catastrophic. The metal level is indicative of the level of coverage the plan
provides.

A plan has a "rate", which is the amount that a consumer pays as a monthly
premium, in dollars.

A plan has a "rate area", which is a geographic region in a state that
determines the plan's rate. A rate area is a tuple of a state and a number, for
example, NY 1, IL 14.

There are two additional CSV files in this directory besides `slcsp.csv`:

- `plans.csv` — all the health plans in the U.S. on the marketplace
- `zips.csv` — a mapping of ZIP code to county/counties & rate area(s)

A ZIP code can potentially be in more than one county. If the county can not be
determined definitively by the ZIP code, it may still be possible to determine
the rate area for that ZIP code. A ZIP code can also be in more than one rate area. In that case, the answer is ambiguous
and should be left blank.

In [135]:
%config Completer.use_jedi = False

In [136]:
import pandas as pd

In [137]:
df_slcsp = pd.read_csv("slcsp.csv", dtype={"zipcode": "string"})
# display(df_slcsp)
df_plans = pd.read_csv("plans.csv")
# display(df_plans)
df_zips = pd.read_csv("zips.csv", dtype={"zipcode": "string"})
# display(df_zips)

In [138]:
df_zip_plan = df_zips.merge(df_plans, how="inner", on=["state", "rate_area"])
# len(df_zip_plan.index)
# df_zip_plan = df_zip_plan.loc[(pd.isna(df_zip_plan.zipcode) == False) & (pd.isna(df_zip_plan.rate) == False)]
# display(df_zip_plan)

In [139]:
df_zip_plan["rank"] = df_zip_plan.groupby("zipcode")["rate"].rank("dense")
# df_zip_plan = df_zip_plan[df_zip_plan["rank"].notnull()]
# df_zip_plan = df_zip_plan[df_zip_plan["rank"] == 2]
display(df_zip_plan)

,zipcode,state,county_code,name,rate_area,plan_id,metal_level,rate,rank
0,36749,AL,1001,Autauga,11,13224PL3852542,Catastrophic,178.79,2.0
1,36749,AL,1001,Autauga,11,03584UW8758085,Silver,268.26,21.0
2,36749,AL,1001,Autauga,11,28130ET1465358,Silver,256.21,16.0
3,36749,AL,1001,Autauga,11,84291ZR3260798,Gold,291.09,28.0
4,36749,AL,1001,Autauga,11,78405DE5161755,Platinum,321.37,32.0
...,...,...,...,...,...,...,...,...,...
2373827,82605,WY,56025,Natrona,1,19256OC0825371,Catastrophic,329.10,3.0
2373828,82605,WY,56025,Natrona,1,04174VX0104821,Gold,452.24,21.0
2373829,82605,WY,56025,Natrona,1,89421LV8282467,Catastrophic,320.14,1.0
2373830,82605,WY,56025,Natrona,1,23064SU4445094,Gold,559.90,42.0


## Cases:
- No matching plan => blank
- One matching plan and hence no second plan => blank
- One matching lowest rate plan and one matching second lowest rate plan => pick the second lowest rate plan
- One matching lowest rate plan and more than one matching second lowest rate plan => blank
- Multiple lowest rate plans have the same rate and no second lowest rate plan => blank
- Multiple lowest rate plans have the same rate and exactly one with the second lowest rate => pick the second lowest rate plan
- Multiple lowest rate plans have the same rate and more than one with the second lowest rate => blank

In [140]:
df_sec_rank = df_zip_plan[df_zip_plan["rank"] == 2]
result =  df_sec_rank.groupby(["zipcode", "rate", "rank"]).size().to_frame("cnt").reset_index()
df_lookup = result[result["cnt"] == 1]
df_not_lookup = result[result["cnt"] != 1]
print(f"df_lookup = {len(df_lookup.index)}")
print(f"df_not_lookup = {len(df_not_lookup.index)}")
print()
display(df_lookup)

df_lookup = 21180
df_not_lookup = 7309



,zipcode,rate,rank,cnt
0,03031,183.63,2.0,1
1,03032,183.63,2.0,1
2,03033,183.63,2.0,1
3,03034,183.63,2.0,1
4,03036,183.63,2.0,1
...,...,...,...,...
28484,97914,161.00,2.0,1
28485,97917,161.00,2.0,1
28486,97918,161.00,2.0,1
28487,97920,161.00,2.0,1


In [141]:
df = df_slcsp[["zipcode"]].merge(df_lookup, on="zipcode", how="left")
display(df)

,zipcode,rate,rank,cnt
0,64148,192.84,2.0,1.0
1,67118,NaN,NaN,NaN
2,40813,NaN,NaN,NaN
3,18229,181.39,2.0,1.0
4,51012,199.48,2.0,1.0
5,79168,190.66,2.0,1.0
6,54923,175.42,2.0,1.0
7,67651,NaN,NaN,NaN
8,49448,NaN,NaN,NaN
9,27702,184.69,2.0,1.0


In [143]:
df = df[["zipcode", "rate"]]
display(df)

,zipcode,rate
0,64148,192.84
1,67118,NaN
2,40813,NaN
3,18229,181.39
4,51012,199.48
5,79168,190.66
6,54923,175.42
7,67651,NaN
8,49448,NaN
9,27702,184.69


In [146]:
df.to_csv("slcsp_solution.csv", index=False)

## Testing

In [149]:
# zip = "46706"
zip = "47387"
display(df_lookup[df_lookup["zipcode"] == zip])
display(df_not_lookup[df_not_lookup["zipcode"] == zip])

,zipcode,rate,rank,cnt


,zipcode,rate,rank,cnt
13376,47387,244.5,2.0,2
